In [ ]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ, ClassicalRegister, QuantumRegister, BasicAer
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.visualization import plot_histogram
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP, NELDER_MEAD
# Loading your IBM Q account(s)
provider = IBMQ.load_account()
provider_1 = IBMQ.get_provider()
import datetime

In [ ]:
#backend = Aer.get_backend('qasm_simulator')
backend = provider_1.get_backend('ibmq_vigo')
time_s = datetime.datetime.now()
N = 8192 #number of shots

def circuit_0(angle):
    q = QuantumRegister(2) 
    c = ClassicalRegister(2) 
    qc = QuantumCircuit(q, c)

    qc.ry(angle[0], q[0])
    qc.rz(angle[1], q[0])
    qc.ry(angle[2], q[1])
    qc.rz(angle[3], q[1])
    qc.cx(q[0], q[1])
    qc.ry(angle[4], q[0])
    qc.rz(angle[5], q[0])
    qc.ry(angle[6], q[1])
    qc.rz(angle[7], q[1])
    
    qc.measure(q[0], c[0])
    qc.measure(q[1], c[1])
    
    job = execute(qc, backend = backend, shots = N)
    result = job.result()
    counts = result.get_counts(qc)
    
    return counts


def sign(matrix0, matrix1):
    matrix = np.kron(matrix0, matrix1)
    diagonal = list(np.diag(matrix))
    return diagonal

def term_value(counts, sign):
    i = 0
    if "00" in list(counts):
        i = i + (sign[0] * counts["00"] / N)
    if "01" in list(counts):
        i = i + (sign[1] * counts["01"] / N)
    if "10" in list(counts):
        i = i + (sign[2] * counts["10"] / N)
    if "11" in list(counts):
        i = i + (sign[3] * counts["11"] / N)
    return i  

counts = []
values = []
def Hamiltonian(angle):
    counts0 = circuit_0(angle)
    counts.append(counts0)
    
    I = np.array([[1, 0], [0, 1]])
    Z = np.array([[1, 0], [0, -1]])
    
    Id = term_value(counts0, sign(I, I)) 
    Z0 = term_value(counts0, sign(I, Z))
    Z1 = term_value(counts0, sign(Z, I))
    Z2 = term_value(counts0, sign(Z, Z))
          
    H = -1.667820214453756 * Id + 0.701945989384994 * Z0 + 0.26392823568376805 * Z1 +  0.701945989384994 * Z2 
    
    values.append(H)
    return H


optimizer = COBYLA(maxiter = 200, tol = 0.001)

angle0 = []
for i in range(0, 8):
    n = random.uniform(- 2 * np.pi, 2 * np.pi)
    angle0.append(n)
    
res = optimizer.optimize(num_vars =  8, objective_function = Hamiltonian, initial_point = angle0)

print(res)

iterations = []
for i in range(1, res[2] + 1):
    iterations.append(i)

print(counts)
    
x = [0, len(values)]
y = [-2.80778396, -2.80778396]

file = open("He_" + str(backend) + "_" + str(len(values)) + ".txt", "w+")
time_e = datetime.datetime.now()
file.write("Backend: " + str(backend) + "\n") 
file.write(str(optimizer.setting) + "\n")
file.write("Time: " + str(time_e - time_s) + "\n") 
file.write("Start: " + str(time_s) + "\n") 
file.write("End: " + str(time_e) + "\n") 
file.write("\n") 
file.write("Initial_point: " + str(angle0) + "\n")
file.write("Results: " + str(res) + "\n")
file.write("\n")
file.write("Iterations" + " " + "Energy [a. u.]" + "\n")
for i in range(0, len(values)):
    file.write(str(iterations[i]) + " " + str(values[i]) + "\n")

file.write("\n")
file.write("Shots: " + str(N) + "\n") 
file.write("Counts: " + str(counts[len(counts) - 1]) + "\n") 
file.close()
